## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Matara,LK,5.9485,80.5353,83.73,70,70,13.00,Clouds
1,1,Albany,US,42.6001,-73.9662,60.03,83,25,2.30,Clouds
2,2,Carnarvon,AU,-24.8667,113.6333,66.27,54,90,5.75,Clouds
3,3,Dikson,RU,73.5069,80.5464,40.96,100,100,11.34,Clouds
4,4,Kapaa,US,22.0752,-159.3190,87.78,69,20,4.00,Clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [14]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                259
City                   259
Country                258
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Current Description    259
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Matara,LK,83.73,Clouds,5.9485,80.5353,
4,Kapaa,US,87.78,Clouds,22.0752,-159.3190,
5,Komsomolskiy,UZ,80.02,Clouds,40.4272,71.7189,
9,Worland,US,78.60,Clouds,44.0169,-107.9554,
10,Waddan,LY,81.93,Clear,29.1614,16.1390,
24,Muslyumovo,RU,76.08,Clouds,55.3033,53.1947,
25,Tarakan,ID,85.50,Clouds,3.3000,117.6333,
28,Alofi,NU,75.09,Rain,-19.0595,-169.9187,
32,Saint George,US,83.62,Clear,37.1041,-113.5841,
33,Atuona,PF,77.20,Clouds,-9.8000,-139.0333,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name. (Pulled from https://www.kite.com/python/answers/how-to-drop-empty-rows-from-a-pandas-dataframe-in-python)
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [27]:
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Matara,LK,83.73,Clouds,5.9485,80.5353,Amaloh Boutique Resort
4,Kapaa,US,87.78,Clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
5,Komsomolskiy,UZ,80.02,Clouds,40.4272,71.7189,Ikathouse
9,Worland,US,78.60,Clouds,44.0169,-107.9554,Town House Motor Inn
10,Waddan,LY,81.93,Clear,29.1614,16.1390,محل الشريف محمد العايب
24,Muslyumovo,RU,76.08,Clouds,55.3033,53.1947,"""Kunakkhana"""
25,Tarakan,ID,85.50,Clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
28,Alofi,NU,75.09,Rain,-19.0595,-169.9187,Taloa Heights
32,Saint George,US,83.62,Clear,37.1041,-113.5841,Best Western Plus Abbey Inn
33,Atuona,PF,77.20,Clouds,-9.8000,-139.0333,Villa Enata


In [28]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))